In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

#### Limpeza e tratamento do conjunto de dados meteorológicos

Fonte dos dados de velocidade do vento: Open-Meteo API
https://open-meteo.com/

Velocidade do vento tomada a uma altura de 100m acima do nível do solo.

Coordenadas utilizadas (Conjunto Eólico Umburanas)
- Latitude: -10.650263
- Longitude: -41.46515

Fonte das coordenadas: ANEEL

Notas:
- A coordenada utilizada na chamada à API não corresponde exatamente ao centróide das coordenadas dos geradores eólicos do conjunto.
- A distância entre os geradores eólicos e a coordenada escolhida varia de 3.5 km e 7.5 km, aproximadamente. Isto só foi percebido pelo autor do estudo após a conclusão do treinamento dos modelos de previsão. O ideal teria sido selecionar um ponto mais próximo aos geradores para diminuir o erro de estimação da velocidade do vento.


In [2]:
# Abrindo o arquivo descarregado da Open-Meteo API
df_tempo = pd.read_csv("..\Dados brutos\OPEN METEO\open-meteo-10.65S41.47W1070m.csv")

In [3]:
df_tempo.head()

,time,temperature_2m (°C),wind_speed_100m (km/h),wind_direction_100m (°),latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,2021-01-01T00:00,22.6,25.0,83,-10.650263,-41.46515,1070.0,0.0,GMT,GMT
1,2021-01-01T01:00,21.4,25.1,74,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01T02:00,20.7,23.8,72,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01T03:00,20.1,23.2,83,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01T04:00,19.5,26.3,100,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Criando uma função para reformatar a coluna que contém as datas e horários
from datetime import datetime
def formatDateTime(date_time):
    date_time = date_time.split("T")
    date, time = date_time
    date_time = date + "\t" + time
    return(datetime.strptime(date_time,"%Y-%m-%d\t%H:%M"))
print(formatDateTime("2021-01-01T01:00"))

2021-01-01 01:00:00


In [5]:
# Renomeando as colunas do DataFrame
df_tempo = df_tempo[["time","temperature_2m (°C)", "wind_speed_100m (km/h)", "wind_direction_100m (°)"]]
df_tempo.rename(columns={
    "time":"data_hora",
    "temperature_2m (°C)":"temperatura",
    "wind_speed_100m (km/h)":"velocidade_vento",
    "wind_direction_100m (°)":"direcao_vento"
}, inplace=True)
df_tempo.head()

,data_hora,temperatura,velocidade_vento,direcao_vento
0,2021-01-01T00:00,22.6,25.0,83
1,2021-01-01T01:00,21.4,25.1,74
2,2021-01-01T02:00,20.7,23.8,72
3,2021-01-01T03:00,20.1,23.2,83
4,2021-01-01T04:00,19.5,26.3,100


In [6]:
df_tempo.describe()

,temperatura,velocidade_vento,direcao_vento
count,24840.000000,24840.000000,24840.000000
mean,20.758321,22.394775,108.060427
std,3.922224,7.114451,42.861160
min,11.300000,0.000000,2.000000
25%,17.800000,18.300000,93.000000
50%,20.200000,23.400000,103.000000
75%,23.600000,27.300000,112.000000
max,33.600000,49.700000,360.000000


Amplitude da temperatura no local selecionado:
    de 11.3°C a 33.6°C
    
Amplitude da velocidade do vento no local selecionado (a uma altura de 100m acima do nível do solo):
    de 0 km/h a 49.7 km/h
    
Direção do vento: de 2° a 360°.

In [7]:
# Reformatando as datas e horários
df_tempo["data_hora"] = df_tempo["data_hora"].apply(formatDateTime)
df_tempo.head()
localidade = "umburanas-ba"

# Salvando os dados formatados
write_path = os.path.join(r"..\Dados tratados","tempo - " + localidade+ ".csv")
df_tempo.to_csv(write_path, encoding = "utf-8", index=False)